# Marine Biodiversity and Environmental Data Exploration with `mlcroissant`
This notebook provides a template for loading and exploring the dataset on marine biodiversity and environmental data using the `mlcroissant` library.

### Dataset Source
The dataset source is provided via a Croissant schema URL.

In [ ]:
# Ensure `mlcroissant` library is installed
!pip install mlcroissant

## 1. Data Loading
Load metadata and records from the dataset using `mlcroissant`.

In [ ]:
import mlcroissant as mlc
import pandas as pd

# Define the dataset URL
url = 'https://dev.senscience.cloud/portal/10.82843/pm80-mh77/fair2.json'

# Load the dataset metadata
dataset = mlc.Dataset(url)
metadata = dataset.metadata.to_json()
print(f"{metadata['name']}: {metadata['description']}")

## 2. Data Overview
Review available record sets, fields, and their IDs.

In [ ]:
for record_set in metadata['recordSet']:
    print(f"RecordSet ID: {record_set['@id']}, Description: {record_set['description']}")
    for field in record_set['field']:
        print(f"  Field ID: {field['@id']}, Name: {field['name']}, DataType: {field['dataType']}")

## 3. Data Extraction
Load data from a specific record set into a DataFrame for analysis. Use the record set and field `@id`s from the overview.

In [ ]:
# Extract data from each record set
record_sets = [
    'https://senscience.ai/frontiers/borja/METADATA.csv',
    'https://senscience.ai/frontiers/borja/WATER.csv',
    'https://senscience.ai/frontiers/borja/BIOTA.csv'
]
dataframes = {}

for record_set in record_sets:
    records = list(dataset.records(record_set=record_set))
    dataframes[record_set] = pd.DataFrame(records)

print(dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'].columns.tolist())
dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'].head()

## 4. Exploratory Data Analysis (EDA)
Apply common data processing steps, such as filtering records based on specific criteria, normalizing numeric fields, and categorizing data. This section should include operations like removing outliers, transforming data distributions, or grouping data by key attributes to prepare it for further analysis.

In [ ]:
# Select a numeric field for analysis
numeric_field = 'https://senscience.ai/frontiers/borja/METADATA.csv/decimallatitude'

threshold = 43.3
filtered_df = dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'][
    dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'][numeric_field] > threshold
]
print(f"Filtered records with {numeric_field} > {threshold}:")
print(filtered_df.head())

filtered_df[f"{numeric_field}_normalized"] = (
    filtered_df[numeric_field] - filtered_df[numeric_field].mean()
) / filtered_df[numeric_field].std()
print(f"Normalized {numeric_field} for filtered records:")
print(filtered_df[[numeric_field, f"{numeric_field}_normalized"]].head())

group_field = 'https://senscience.ai/frontiers/borja/METADATA.csv/taxagroup'
if group_field in dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'].columns:
    grouped_df = filtered_df.groupby(group_field).mean()
    print(f"Grouped data by {group_field}:")
    print(grouped_df.head())

## 5. Visualization
Visualize data distributions or relationships between fields in the dataset.

In [ ]:
import matplotlib.pyplot as plt

# Plot the distribution of a numeric field
plt.figure(figsize=(10, 6))
plt.hist(dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'][numeric_field], bins=30, alpha=0.7)
plt.title('Distribution of Decimal Latitude')
plt.xlabel('Decimal Latitude')
plt.ylabel('Frequency')
plt.show()

## 6. Conclusion
Summarize key findings and observations from the dataset exploration.
- The dataset provides extensive information about marine biodiversity and environmental monitoring.
- Key fields such as latitude and taxonomic group help identify sampling sites and organism classifications.
- The analysis shows the potential for in-depth exploration of environmental parameters and their spatial distribution.